In [2]:
import gurobipy as gp

In [28]:
phi = {0:1, 3:1}
optimal  = dict()
z = []
for keys in phi.keys():
    z.append(keys)

for i in range(10):
    try:
        j = 0
        while j + 1 < len(z):
            m = gp.Model()
            x = m.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, name = 'x')
            y = m.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, name = 'y')
        # Enter constraints and objective functions
            constr_lower = [x-y >= 1, y <= 0, x+y <= 2]     ############################
            constr_upper = [x >= 0, x <= 3]                 ############################# 
            F_upper = (x-1.5)**2 + (y+1)**2                 #############################
            f_lower = -y                                    #############################
            for cs in constr_lower:
                m.addConstr(cs)
            for cs in constr_upper:
                m.addConstr(cs)
#lagrangian = mu_1*phi[z[0]] + mu_2*phi[z[1]] - lambda_1*mu_1 - lambda_2*mu_2 + beta*(mu_1 + mu_2 - 1) + gamma*(mu_1*z[0] + mu_2*z[1])
            mu_1 = m.addVar(name = "mu_1", ub = gp.GRB.INFINITY)
            mu_2 = m.addVar(name = "mu_2", ub = gp.GRB.INFINITY)
            lambda_1 = m.addVar(lb = 0, ub = gp.GRB.INFINITY)
            lambda_2 = m.addVar(lb = 0, ub = gp.GRB.INFINITY)
            beta = m.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY)
            gamma = m.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY)
            m.addConstr(phi[z[j]] - lambda_1 + beta + gamma*z[j] == 0)
            m.addConstr(phi[z[j+1]] - lambda_2 + beta + gamma*z[j+1] == 0)
            m.addConstr(mu_1 + mu_2 == 1)
            m.addConstr(mu_1*z[j] + mu_2*z[j+1] == x)
            m.addConstr(f_lower <= mu_1*phi[z[j]] +mu_2*phi[z[j+1]])
            m.setObjective(F_upper, gp.GRB.MINIMIZE)
            m.optimize()
            F_upper_value = F_upper.getValue()
            f_lower_value = f_lower.getValue()                  # Value of f at the new point
            z.append(x.X)
#Calculate Phi
            model = gp.Model()
            x = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, name = 'x')
            y = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, name = 'y')
# Enter constraints and objective functions
            constr_lower = [x-y >= 1, y <= 0, x+y <= 2]
            constr_upper = [x >= 0, x <= 3]
            F_upper = (x-1.5)**2 + (y+1)**2
            f_lower = -y
            for cs in constr_lower:
                model.addConstr(cs)
            for cs in constr_upper:
                model.addConstr(cs)
            model.addConstr(x >= z[j])
            model.addConstr(x <= z[j+1])
            x = z[len(z)-1]
            model.setObjective(f_lower, gp.GRB.MINIMIZE)
            model.optimize()
#Use optimal objective
            phi_value = f_lower.getValue()                           #Value of Phi at the new point
# Append to dictionary and sort dictionar and the array z
            phi[x] = phi_value
# Also add termination condition given in the paper
            if (phi_value == f_lower_value):
                optimal[(x, y.X)] = F_upper_value
        
            phi = dict(sorted(phi.items()))
            z.sort()
#j+2. j+3
            j = j + 2
    except Exception:
        continue

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 10 rows, 8 columns and 19 nonzeros
Model fingerprint: 0x53e58869
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 3e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 8 rows and 6 columns
Presolve time: 0.06s
Presolved: 2 rows, 2 columns, 4 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.00501250e+05 -5.03668250e+05  2.00e+03 3.35e+02  7.23e+05     0s
   1   2.07866471e+04 -2.66510017e+0

In [29]:
phi

{0: 1.0,
 3.5245783413480456e-14: 0.22170818692688132,
 0.7782918130731187: -0.0,
 1.0384615359642075: -0.0,
 1.500000000057013: -0.0,
 1.9615384704786938: -0.0,
 2.221708200864027: -0.0,
 3: 1,
 3.0000000000000173: 0.2217082008640272}

In [30]:
optimal

{(1.500000000057013, 0.0): 1.0,
 (1.0384615359642075, 0.0): 1.2130177537845186,
 (1.9615384704786938, 0.0): 1.2130177597318115,
 (0.0, -1.0): 2.25}